In [ ]:
import os.path as path
import sys
import numpy as np
from pyspark import SparkContext, SparkConf
from lib import *
from pyspark.sql import *
# from utils import *
from lib.utils import *
import findspark
findspark.init()

%pylab inline

import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import branca.colormap as cm



def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json',
    'us-states.json',
     json.load)


new_geo = []
for i, s in enumerate(geo_json_data['features']):
    if s['id'] in states:
        new_geo.append(s)      

geo_json_data['features'] = new_geo

In [ ]:
# Create a new spark and sql context
sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])
sqlContext = SQLContext(sc)

# Create the names of the files/directories we're working with
data_dir = '../DataHW3'

if not path.exists(data_dir + '/' + 'stations.parquet'):
    getStations()

    
states = ['ND', 'SD', 'MN', 'IA', 'NE', 'TX', 'OK', 'KS']

for s in states:
    
    parquet = s + '.parquet'
    tarname = s + '.tgz'
    
    if not path.exists(data_dir + '/' + parquet):

        # pull the weather data for a particular state from the MAS-DSE S3 bucket
        getStateData(s, data_dir, tarname, parquet)

# Change in average daily snow fall

In [ ]:
states = ['ND', 'SD', 'MN', 'IA', 'NE', 'TX', 'OK', 'KS']

featureStr = "\'SNOW\'"

test_data = decadeMeasurementDelta_states(featureStr, states, data_dir, sqlContext)

master_dataframe = test_data[0]
for i in range(len(test_data)-1):
    master_dataframe = master_dataframe.union(test_data[i+1])


In [ ]:
sqlContext.registerDataFrameAsTable(master_dataframe, 'table')
Query = f"""
SELECT state, COUNT(Station), MEAN(Values)
FROM table
GROUP BY state
"""
dframe = sqlContext.sql(Query).toPandas()
dframe.count()

In [ ]:
clrmap=cm.LinearColormap(colors=['blue', 'lightblue', 'white', 'lightcoral', 'red'], index=[-50,-25, 0,25,50],vmin=-50, vmax=50)

In [ ]:
snow = dict(zip(list(dframe['state']), list(dframe['avg(Values)'])))
print(snwd)

layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=snow,
    colormap=clrmap,
    border_color='black',
    style={'fillOpacity': 0.8})
del(m)
m = ipyleaflet.Map(center = (43,-100), zoom = 4)
m.add_layer(layer)

## SNOW - 70s vs 00s

In [ ]:
m

In [ ]:
clrmap

# SNWD

In [ ]:
featureStr = "\'SNWD\'"

test_data = decadeMeasurementDelta_states(featureStr, states, data_dir, sqlContext)

master_dataframe = test_data[0]
for i in range(len(test_data)-1):
    master_dataframe = master_dataframe.union(test_data[i+1])


In [ ]:
sqlContext.registerDataFrameAsTable(master_dataframe, 'table')
Query = f"""
SELECT state, COUNT(Station), MEAN(Values)
FROM table
GROUP BY state
"""
dframe = sqlContext.sql(Query).toPandas()
dframe.count()

In [ ]:
clrmap=cm.LinearColormap(colors=['blue', 'lightblue', 'white', 'lightcoral', 'red'], index=[-50,-25, 0,25,50],vmin=-50, vmax=50)

In [ ]:
snwd = dict(zip(list(dframe['state']), list(dframe['avg(Values)'])))
print(snwd)

layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=snwd,
    colormap=clrmap,
    border_color='black',
    style={'fillOpacity': 0.8})
del(m)
m = ipyleaflet.Map(center = (43,-100), zoom = 4)
m.add_layer(layer)

In [ ]:
m

In [ ]:
clrmap